In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 5.6 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


In [3]:
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard, polite version to one that offers a bit more encouragement. Choose the one that best fits your company's culture and the stage of the hiring process.

---

**Option 1: Standard & Polite (Most Common)**

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to [interview with our team / submit your application].

We appreciate you sharing your qualifications and experience with us. We received a large number of applications for this role, and after careful consideration, we have decided to move forward with other candidates whose qualifications and experience were a closer match for the specific requirements of this role at this time.

This was a very competitive search, and we truly appreciate the time and effort you invested in the process.

We wish you t

In [5]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Vidhya.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Dear Vidhya,

Thank you for your interest in RocketBoots. We appreciate you taking the time to apply.

While we were impressed, the requirements for this role have recently changed. We wish you the very best in your job search and future endeavors.

Keep flying,
RocketBoots HR


In [6]:
structured_prompt = """
# Context
You are a Senior Python Developer with expertise in writing clean, production-quality code.

# Objective
Write a Python function that reverses a given string.

# Constraints
1. The function must use recursion.
2. Do NOT use slicing (e.g., [::-1]).
3. Do NOT use any built-in reverse functions.

# Style Requirements
1. Include detailed docstrings explaining:
   - Parameters
   - Return value
   - How the recursion works
2. Follow PEP 8 conventions.
3. Keep the implementation clean and readable.

# Output Format
Return only valid Python code.
"""


In [7]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

In [8]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: The perfect word for that is **Hangry**!


In [9]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Output: Hangry (The completely valid excuse for becoming an irrational monster until someone shoves food in your face.)


In [10]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

The application is not performing to optimal standards.
